# Day 07

https://adventofcode.com/2022/day/7

In [ ]:
import aocd

day, year = 7, 2022

## Input

In [ ]:
check_example = True
example = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

data = example if check_example else aocd.get_data(day=day, year=year)

In [ ]:
from __future__ import annotations

from dataclasses import dataclass, field
from typing import Union, List, Dict

In [ ]:
@dataclass
class File:
    name: str
    size: int
    path: str
        
    def __repr__(self):
        return f'{self.size} {self.name}\n'

@dataclass
class Directory:
    path: str
    parent: Union[Directory, None]
    files: Dict[str, File] = field(default_factory=dict)
    dirs: Dict[str, Directory] = field(default_factory=dict)

    def add_dir(self, path: str):
        if path not in self.dirs.keys():
            self.dirs[path] = Directory(self.path + path + '/', parent=self)
            
    def add_file(self, path: str, size: Union[str, int]):
        self.files[path] = File(path, int(size), self.path + path)
    
    @property
    def depth(self):
        return self.path.count('/') 

    @property
    def size(self):
        file_sizes = sum([f.size for f in self.files.values()])
        dir_sizes = sum([d.size for d in self.dirs.values()])
        return file_sizes + dir_sizes
    
    def walk_files(self):
        for f in self.files.values():
            yield f
        for d in self.dirs.values():
            yield from d.walk_files()

    def walk_dirs(self):
        for d in self.dirs.values():
            yield d
            yield from d.walk_dirs()
            
    def __repr__(self):
        str_repr = f'{self.path}\n'
        for d in self.dirs.values():
            str_repr += (self.depth * '  ') + repr(d)
        for f in self.files.values():
            str_repr += (self.depth * '  ') + repr(f)
        return str_repr

In [ ]:
root = Directory('/', parent=None)

In [ ]:
current_dir = root
for l in data.splitlines():
    if '$ cd' in l:
        _, cmd, path = l.partition('$ cd ')
        if path.strip() == '/': 
            current_dir = root
        elif path == '..':
            current_dir = current_dir.parent
        else:
            current_dir.add_dir(path)
            current_dir = current_dir.dirs[path]
        next
    elif '$ ls' in l:
        next
    elif 'dir ' in l:
        _, path = l.split(' ')
        current_dir.add_dir(path)
        next
    else:
        size, path = l.split(' ')
        current_dir.add_file(path, size)

In [ ]:
root

/
  /a/
    /a/e/
      584 i
    29116 f
    2557 g
    62596 h.lst
  /d/
    4060174 j
    8033020 d.log
    5626152 d.ext
    7214296 k
  14848514 b.txt
  8504156 c.dat

## Part 1

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

In [ ]:
part1 = sum([d.size for d in root.walk_dirs() if d.size <= 100000])

print("Part 1:", part1)

Part 1: 95437


In [ ]:
if not check_example:
    aocd.submit(part1, part=1, day=day, year=year)

## Part 2 

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?

In [ ]:
min_space2delete = 30000000 - (70000000 - root.size)

In [ ]:
part2 = min([d.size for d in root.walk_dirs() if d.size >= min_space2delete])

print("Part 2:", part2)

Part 2: 24933642


In [ ]:
if not check_example:
    aocd.submit(part2, part=2, day=day, year=year)